<a href="https://colab.research.google.com/github/Kaiziferr/deep_learning_workshop/blob/master/01_workshop_forecasting_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install h5py

In [2]:
from keras.models import Sequential, model_from_json
from keras.layers import Dense
import pandas as pd
from keras.models import load_model

# **Data**

---



In [3]:
url = 'https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_keras_sklearn_introduction/diabetes.csv'
data = pd.read_csv(url)
data.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [4]:
X = data.drop('Outcome', axis = 1)
y = data['Outcome']

# **Model**

---



In [5]:
model = Sequential()
model.add(Dense(12, input_dim = 8, kernel_initializer='glorot_uniform', activation = 'relu'))
model.add(Dense(8, kernel_initializer = 'glorot_uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid'))

#Compile Model
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

#Ajuste
model.fit(X, y, epochs = 150, batch_size = 10, verbose = 0)

score = model.evaluate(X, y, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

accuracy: 65.36%


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                108       
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


# **Serializado JSON**

---



In [7]:
model_json = model.to_json()

In [8]:
model_json

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 8], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_re

In [9]:
with open('model.json', 'w') as f:
  f.write(model_json)

# **Serializado pesos y topologia con HDF5**

---



In [10]:
model.save_weights('model.h5')

In [11]:
json_file = open('model.json', 'r')
load_model_json = json_file.read()
json_file.close

load_model_json = model_from_json(load_model_json)

In [12]:
load_model_json.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                108       
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [13]:
load_model_json.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
print('model cargado')

model cargado


In [14]:
score = load_model_json.evaluate(X, y, verbose = 0)
print("%s: %.2f%%" % (load_model_json.metrics_names[1], score[1]*100))

accuracy: 65.10%


# **Forecasting**

---



In [15]:
# Guardar topologias y pesos
model.save('model_estructura.h5')

In [16]:
model = load_model('model_estructura.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                108       
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [17]:
score = model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

accuracy: 65.36%
